# Prototypical UI Strategy 1

## 0. Prerequesites - Installing of packages and Loading Files

In [ ]:
import shap
import pickle
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
import toml
import os
os.chdir("..")
config = toml.load("config.toml")
from src.utils import to_prob_list
from src.utils import *

import toml
import sys

from sklearn.metrics import confusion_matrix
from src.preparation_pipeline import PreparationPipeline
from src.explain_pipeline import ExplainPipeline
from src.explainability_pipeline import ExplainabilityPipeline
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from src.utils import plot_confusion_matrix

import ipywidgets as widgets
from ipywidgets import AppLayout
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
from ipywidgets import AppLayout, Button
from IPython.display import display, clear_output
from IPython.display import display

from matplotlib_venn import venn2, venn3, venn3_circles

import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import Button, GridBox, Layout, ButtonStyle

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Layout, Button, Box
%matplotlib inline

In [ ]:
base_path = config["result"]["results"]
strategy = "strategy 1/"
strengths = ["impure_0.0", "impure_0.1", "impure_0.2", "impure_0.3","impure_0.4","impure_0.5","impure_0.6","impure_0.7","impure_0.8","impure_0.9","impure_1.0"]
strength = 3

In [ ]:
with open(base_path + strategy + "1_bert_shap_values_" + strengths[strength] + ".pckl", 'rb') as f:
     shap_values_bert = pickle.load(f)
        
with open(base_path + strategy + "1_bert_logits_" + strengths[strength] + ".pckl", 'rb') as f:
     logits_bert = pickle.load(f)

with open(base_path + strategy + "1_xgb_shap_values_" + strengths[strength] + ".pckl", 'rb') as f:
     shap_values_xgb = pickle.load(f)
        
with open(base_path + strategy + "1_xgb_class_proba_" + strengths[strength] + ".pckl", 'rb') as f:
     class_proba_xgb = pickle.load(f)

with open(base_path + "results_paper/strategy 1/" + "1_labels_combined_" + strengths[strength] + ".pckl", 'rb') as f:
     labels_combined = pickle.load(f)
        
with open(base_path + strategy + "1_xgb_y_test_" + strengths[strength] + ".pckl", 'rb') as f:
     y_test = pickle.load(f)

In [ ]:
logits_bert = logits_bert.tolist()
logits_bert = list(map(to_prob_list, logits_bert))
bert_class = np.argmax(logits_bert, axis=1) 
xgb_class = np.argmax(class_proba_xgb, axis=1) 

confusion_matrix_xgb = confusion_matrix(y_test, xgb_class)
confusion_matrix_bert = confusion_matrix(y_test, bert_class)
confusion_matrix_combined = confusion_matrix(y_test, labels_combined[1])

## 1. UI Widget Definition

In [ ]:
plt.rcParams["figure.dpi"] = 300
non_text_output_l = widgets.Output()
non_text_output_g1 = widgets.Output()
non_text_output_g2 = widgets.Output()
text_output_l1 = widgets.Output()
text_output_l2 = widgets.Output()
text_output_g1 = widgets.Output()
text_output_g2 = widgets.Output()
output_model_metrics_non_text = widgets.Output()
output_model_metrics_text = widgets.Output()
output_model_metrics_result = widgets.Output()
output_model_num_metrics_non_text = widgets.Output()
output_model_num_metrics_text = widgets.Output()
output_model_num_metrics_result = widgets.Output()
pred_t_output = widgets.Output()
true_t_output = widgets.Output()
pred_nt_output = widgets.Output()
true_nt_output = widgets.Output()

w = widgets.IntSlider(value=15,
    min=0,
    max=400,
    step=1,
    description='Case:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d')

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%', height='50%')

box_layout_100 = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%', height='100%')

with pred_t_output:
    clear_output()
    out = "Rejected"
    if bert_class[w.value] == 1:
        out = "Accepted"
    print("Predicted Label: " + out)
    print("")
    
with true_t_output:
    clear_output()
    out = "Rejected"
    if y_test[w.value] == 1.0:
        out = "Accepted"
    print("True Label: " + out)

    
with pred_nt_output:
    clear_output()
    out = "Rejected"
    if xgb_class[w.value] == 1:
        out = "Accepted"
    print("Predicted Label: " + out)
    print("")
    
with true_nt_output:
    clear_output()
    out = "Rejected"
    if y_test[w.value] == 1.0:
        out = "Accepted"
    print("True Label: " + out)

with text_output_l1:
    clear_output()
    shap.force_plot(shap_values_bert[w.value],  matplotlib=True)
with text_output_l2:
    clear_output()
    shap.plots.text(shap_values_bert[w.value], cmax=0.3)    
    plt.show()
with non_text_output_l:
    clear_output()
    shap.plots.waterfall(shap_values_xgb[w.value], max_display=10,  show=False)
    plt.show()
with non_text_output_g1:
    clear_output()
    shap.plots.bar(shap_values_xgb, max_display=12,  show=False)
    plt.show()
with non_text_output_g2:
    clear_output()
    shap.plots.beeswarm(shap_values_xgb,  show=False)
    plt.show()
with text_output_g1:
    clear_output()
    shap.plots.bar(shap_values_bert,  max_display=12,  show=False)
    plt.show()
with text_output_g2:
    clear_output()
    plt = n_important_words_wordcloud(shap_values_bert,100)
    plt.show()
    
with output_model_metrics_non_text:
    clear_output()
    plt.show(plot_confusion_matrix(confusion_matrix_xgb))
    
with output_model_metrics_text:
    clear_output()
    plt.show(plot_confusion_matrix(confusion_matrix_bert))

with output_model_metrics_result:
    clear_output()
    plt.show(plot_confusion_matrix(confusion_matrix_combined))
    
with output_model_num_metrics_non_text:
    clear_output()
    #print(classification_report(y_test, xgb_class))
    print("ROC AUC Score: " + str(round(roc_auc_score(y_test, xgb_class),2)))
    print("F1-Score:      " + str(round(f1_score(y_test, xgb_class),2)))
    print("Accuracy:      " + str(round(accuracy_score(y_test, xgb_class),2)))
    
with output_model_num_metrics_text:
    clear_output()
    #print(classification_report(y_test, bert_class))
    print("ROC AUC Score: " + str(round(roc_auc_score(y_test, bert_class),2)))
    print("F1-Score:      " + str(round(f1_score(y_test, bert_class),2)))
    print("Accuracy:      " + str(round(accuracy_score(y_test, bert_class),2)))

with output_model_num_metrics_result:
    clear_output()
    #print(classification_report(y_test, labels_combined[1]))
    print("ROC AUC Score: " + str(round(roc_auc_score(y_test, labels_combined[1]),2)))
    print("F1-Score:      " + str(round(f1_score(y_test, labels_combined[1]),2)))
    print("Accuracy:      " + str(round(accuracy_score(y_test, labels_combined[1]),2)))

def on_value_change(change):
    with text_output_l1:
        clear_output()
    with text_output_l2:
        clear_output()
    with non_text_output_l:
        clear_output()
    with text_output_l1:
        shap.force_plot(shap_values_bert[w.value],  matplotlib=True)
    with text_output_l2:
        shap.plots.text(shap_values_bert[w.value], cmax=0.3)
    with non_text_output_l:
        shap.plots.waterfall(shap_values_xgb[w.value])
    with pred_t_output:
        clear_output()
        out = "Rejected"
        if bert_class[w.value] == 1:
            out = "Accepted"
        print("Predicted Label: " + out)
        print("")
    with true_t_output:
        clear_output()
        out = "Rejected"
        if y_test[w.value] == 1.0:
            out = "Accepted"
        print("True Label: " + out)
    with pred_nt_output:
        clear_output()
        out = "Rejected"
        if xgb_class[w.value] == 1:
            out = "Accepted"
        print("Predicted Label: " + out)
        print("")
    with true_nt_output:
        clear_output()
        out = "Rejected"
        if y_test[w.value] == 1.0:
            out = "Accepted"
        print("True Label: " + out)
    
w.observe(on_value_change, names='value')

In [ ]:
box1 = Box(children=[text_output_g1], layout=box_layout)
box2 = Box(children=[text_output_g2], layout=box_layout)
box10 = Box(children=[true_t_output], layout=box_layout_100)
box11 = Box(children=[pred_t_output], layout=box_layout_100)
box12 = Box(children=[true_nt_output], layout=box_layout_100)
box13 = Box(children=[pred_nt_output], layout=box_layout_100)


true_label  = Button(description="True Label",
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

prediction  = Button(description="Predicted Label",
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

shap_bar_plot  = Button(description='SHAP Bar Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))
shap_force_plot  = Button(description='SHAP Force Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

shap_beeswarm_plot  = Button(description='SHAP Beeswarm Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

wordcloud = Button(description='Wordcloud 100 Most Important Words',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

shap_text_plot = Button(description='SHAP Text Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

shap_waterfall_plot = Button(description='SHAP Waterfall Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))


text_g = widgets.Accordion([VBox(children=[ shap_bar_plot,
                                     box1,wordcloud, box2])],layout=Layout(width='50%'))
text_g.set_title(0, 'Textual Model')

box3 = Box(children=[non_text_output_l], layout=box_layout_100)


non_text_l = widgets.Accordion([VBox(children=[true_label, box12, prediction, box13, shap_waterfall_plot,
                                     box3])], layout=Layout(width='50%'))

non_text_l.set_title(0, 'Non-Textual Model')

box4 = Box(children=[text_output_l1], layout=box_layout_100)
box5 = Box(children=[text_output_l2], layout=box_layout_100)

text_l = widgets.Accordion([VBox(children=[true_label, box10, prediction, box11, shap_force_plot,
                                     box4, shap_text_plot, box5])], layout=Layout(width='50%'))
text_l.set_title(0, 'Textual Model')

box6 = Box(children=[non_text_output_g1], layout=box_layout_100)
box7 = Box(children=[non_text_output_g2], layout=box_layout_100)


non_text_g = widgets.Accordion([VBox(children=[  shap_bar_plot,
                                     box6, shap_beeswarm_plot, box7])], layout=Layout(width='50%'))
non_text_g.set_title(0, 'Non-Textual Model')

metrics_text = widgets.Accordion([output_model_metrics_text], layout=Layout(width='33%'))
metrics_text.set_title(0, 'Textual Model Confusion Matrix')

metrics_non_text = widgets.Accordion([output_model_metrics_non_text], layout=Layout(width='33%') )
metrics_non_text.set_title(0, 'Non-Textual Model Confusion Matrix')

metrics_result = widgets.Accordion([output_model_metrics_result], layout=Layout(width='33%'))
metrics_result.set_title(0, 'Resulting Model Confusion Matrix')

num_metrics_text = widgets.Accordion([output_model_num_metrics_text], layout=Layout(width='33%'))
num_metrics_text.set_title(0, 'Textual Model Metrics')

num_metrics_non_text = widgets.Accordion([output_model_num_metrics_non_text], layout=Layout(width='33%') )
num_metrics_non_text.set_title(0, 'Non-Textual Model Metrics')

num_metrics_result = widgets.Accordion([output_model_num_metrics_result], layout=Layout(width='33%'))
num_metrics_result.set_title(0, 'Resulting Model Metrics')

tab1 = VBox(children=[HBox(children=[
                                     non_text_g, text_g]),                  
                      ])
                                     
tab2 = VBox(children=[VBox(children=[w,HBox(children=[
                                     non_text_l, text_l]),
                             ])
                      ])

tab3 = VBox(children=[HBox(children=[ 
                                     metrics_non_text,metrics_text,  metrics_result]), HBox(children=[ 
                                      num_metrics_non_text,num_metrics_text, num_metrics_result]),
                                     ])
tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Global Explainability')
tab.set_title(1, 'Local Explainability')
tab.set_title(2, 'Model Metrics')

## 2. UI Widget Presentation

In [ ]:
VBox(children=[tab])